Setup imports

In [39]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Reading training data

In [3]:
train_df = pd.read_parquet("./data/yellow_tripdata_2022-01.parquet")

In [7]:
train_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [22]:
len(train_df)

2463931

Q1: How many columns are there?

In [6]:
len(train_df.columns)

19

Calculating Duration

In [12]:
train_df['duration'] = train_df['tpep_dropoff_datetime'] - train_df['tpep_pickup_datetime']
train_df['duration'] = train_df['duration'].apply(lambda duration: duration.total_seconds() / 60)

Q2: What's the standard deviation of the trips duration in January?

In [14]:
np.std(train_df['duration'])

46.445295712725304

Q3: What fraction of the records left after you dropped the outliers?

In [26]:
train_df = train_df[(train_df.duration >= 1) & (train_df.duration <= 60)]
len(train_df)

2421440

In [25]:
2421440 / 2463931

0.9827547930522406

Q4: What's the dimensionality of this matrix (number of columns)?

In [36]:
dv = DictVectorizer()

train_dict = train_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
x_train = dv.fit_transform(train_dict)

In [38]:
x_train.get_shape()

(2421440, 515)

Q5: What's the RMSE on train?

In [40]:
lr = LinearRegression()
y_train = train_df['duration']

lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

np.sqrt(mean_squared_error(y_train, y_pred))


6.9861908315536585

Validation data setup

In [44]:
val_df = pd.read_parquet("./data/yellow_tripdata_2022-02.parquet")

val_df['duration'] = val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']
val_df['duration'] = val_df['duration'].apply(lambda duration: duration.total_seconds() / 60)

val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

val_dict = val_df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
x_val = dv.transform(val_dict)

y_val = val_df['duration']

Q6: What's the RMSE on validation?

In [45]:
val_pred = lr.predict(x_val)

np.sqrt(mean_squared_error(y_val, val_pred))


7.786409148550689